In [1]:
# Importing libraries
import json
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch import cuda
device = 'cuda:1' if cuda.is_available() else 'cpu'

In [2]:
from TrainKeywords import YourDataSetClass, model_params

In [91]:
gen_key = final_df['Generated Text'][0]

In [98]:
temp = gen_key.split('Keywords: ')
for t in temp[1:]:
    kw_str = t.split('",')[0]
    keywords
    print(keywords)

[spent, light, considered]
[darkness, twilight, dark]
[day, day, bright]
[fog, white, clouds]
[night, sun, pale]
[wind, thine, earth]
[death, life, long]
[remain, time, past]
[time, pass, distant]
[misplaced, days, years]
[blown, night, wind]
[heat, cold, air]
[rain, water, falls]
[water, black, dust]
[


In [35]:
from transformers import LogitsProcessorList,MinLengthLogitsProcessor,BeamSearchScorer

In [69]:
text = "Generate storyline from title: " + title
encoder_input_ids = tokenizer.encode_plus(text, max_length=30,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt")
encoder_input_ids = encoder_input_ids["input_ids"].to(device, dtype = torch.long)
# lets run beam search using 3 beams
num_beams = 2
# define decoder start token ids
input_ids = torch.ones((num_beams, 1), device=model.device, dtype=torch.long)
input_ids = input_ids * model.config.decoder_start_token_id
# add encoder_outputs to model keyword arguments
model_kwargs = {
"encoder_outputs": model.get_encoder()(encoder_input_ids.repeat_interleave(num_beams, dim=0), return_dict=True)}


In [71]:
# instantiate beam scorer
beam_scorer = BeamSearchScorer(
batch_size=1,
max_length=100,
num_beams=num_beams,
device=model.device)
# instantiate logits processors
logits_processor = LogitsProcessorList([
MinLengthLogitsProcessor(1, eos_token_id=model.config.eos_token_id),])
outputs = model.beam_search(input_ids, beam_scorer, logits_processor=logits_processor, **model_kwargs, max_length=30, return_dict_in_generate = True, output_scores = True)
print("Generated:", tokenizer.batch_decode(outputs['sequences'], skip_special_tokens=True))

Generated: ["'Paris', 'August','sun'], ['sun','sun',"]


In [67]:
outputs['sequences_scores']

tensor([-0.6677], device='cuda:2')

In [114]:
import pronouncing

In [139]:
pronouncing.rhymes('bade')

['abrade',
 'ade',
 'afraid',
 'aid',
 'aide',
 'allayed',
 'arcade',
 'arrayed',
 'bayed',
 'betrayed',
 'blade',
 'blockade',
 'braid',
 'brascade',
 'brigade',
 'brigade',
 'brocade',
 'buffeted',
 'cade',
 'cascade',
 'charade',
 'cliched',
 'colonnade',
 'conveyed',
 'crocheted',
 'crusade',
 'dade',
 'decade',
 'decayed',
 'degrade',
 'delayed',
 'dismayed',
 'disobeyed',
 'displayed',
 'dissuade',
 'downgrade',
 'evade',
 'evade',
 'eyde',
 'fade',
 'fayed',
 'filleted',
 'flayed',
 'forbade',
 'frayed',
 'gade',
 'glade',
 'goldade',
 'grade',
 'granade',
 'grenade',
 'hade',
 'haid',
 'handmade',
 'handmade',
 'heyd',
 'heyde',
 'home-made',
 'homemade',
 'homemade',
 'invade',
 'jade',
 'kade',
 'kincade',
 'kincaid',
 'lade',
 'laid',
 'lemonade',
 'made',
 'maid',
 'manmade',
 'marcade',
 'masquerade',
 'mayde',
 'mccade',
 'mcdade',
 'mcdaid',
 'mcglade',
 'mcquade',
 'mcquaid',
 'mcquaide',
 'metrodade',
 'mislaid',
 'obeyed',
 'okayed',
 'outweighed',
 'overpaid',
 'over

In [140]:
pronouncing.phones_for_word('sympathy')

['S IH1 M P AH0 TH IY0']

In [16]:
title = '''No One Goes to Paris in August'''

text =  "Generate storyline from title: " + title

encoding = tokenizer.encode_plus(text, max_length=30,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt")
input_ids = encoding["input_ids"].to(device, dtype = torch.long)
attention_masks = encoding["attention_mask"].to(device, dtype = torch.long)


generated_ids = model.generate(
              input_ids = input_ids,
              attention_mask = attention_masks, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0,
              early_stopping=True
              )
preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
print(text, preds, sep = '\t')

Generate storyline from title: No One Goes to Paris in August	["'Paris', 'August', 'goes'], ['Sweet','summer', 'brings'], ['Beauty', 'beauty','makes'], ['Past', 'past', 'future'], ['Father', 'Goes', 'tomorrow'], ['Mother', 'loves', 'women'], ['Woman', 'heart', 'dear'], ['Till', '"]
